Dataset from: https://www.kaggle.com/datasets/chebotinaa/fast-food-marketing-campaign-ab-test

**About Dataset:**

A fast-food chain plans to add a new item to its menu. 
However, they are still undecided between three possible marketing campaigns for promoting the new product. 
In order to determine which promotion has the greatest effect on sales, the new item is introduced at locations in several randomly selected markets. 
A different promotion is used at each location, and the weekly sales of the new item are recorded for the first four weeks.

**Goal:**
Evaluate A/B testing results and decide which marketing strategy works the best.


**Columns:**

**MarketID:** unique identifier for market 

**MarketSize:** size of market area by sales 

**LocationID:** unique identifier for store location 

**AgeOfStore:** age of store in years 

**Promotion:** one of three promotions that were tested 

**week:** one of four weeks when the promotions were run 

**SalesInThousands** sales amount for a specific LocationID, Promotion, and week


## 1) Importing Libraries and Data

In [10]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import levene

!pip install pingouin
import pingouin as pg

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------------------------------ --------- 1.0/1.4 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 5.5 MB/s eta 0:00:00


In [12]:
market_df=pd.read_csv("C:/Users/New PC 3/Desktop/DAT25/AB/fast food marketing campaign ab/WA_Marketing-Campaign.csv")

In [14]:
market_df.head()

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands
0,1,Medium,1,4,3,1,33.73
1,1,Medium,1,4,3,2,35.67
2,1,Medium,1,4,3,3,29.03
3,1,Medium,1,4,3,4,39.25
4,1,Medium,2,5,2,1,27.81


In [16]:
market_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MarketID          548 non-null    int64  
 1   MarketSize        548 non-null    object 
 2   LocationID        548 non-null    int64  
 3   AgeOfStore        548 non-null    int64  
 4   Promotion         548 non-null    int64  
 5   week              548 non-null    int64  
 6   SalesInThousands  548 non-null    float64
dtypes: float64(1), int64(5), object(1)
memory usage: 30.1+ KB


In [18]:
market_df.groupby(["Promotion", "week"]).agg({"SalesInThousands":"mean"})

SalesInThousands
Promotion week                  
1         1            58.244419
          2            56.929535
          3            58.774884
          4            58.447209
2         1            47.730213
          2            47.582553
          3            47.722128
          4            46.282766
3         1            55.776170
          2            55.949149
          3            54.377872
          4            55.354681

## 2) Exploratory Analysis

In [20]:
market_df.groupby(["MarketSize", "week"]).agg({"SalesInThousands":"mean"})

SalesInThousands
MarketSize week                  
Large      1            70.793333
           2            68.656667
           3            70.647381
           4            70.369524
Medium     1            44.403875
           2            44.378125
           3            44.107500
           4            43.051875
Small      1            56.245333
           2            58.675333
           3            55.348667
           4            59.368000

In [44]:
#Interestingly, the large and small market sizes give higher gains compared to medium sized stores

In [46]:
market_df.groupby(["MarketSize", "Promotion","week"]).agg({"SalesInThousands":"mean"})

SalesInThousands
MarketSize Promotion week                  
Large      1         1            75.042143
                     2            73.215714
                     3            75.474286
                     4            77.211429
           2         1            60.405000
                     2            59.354375
                     3            61.741250
                     4            59.787500
           3         1            79.687500
                     2            75.740833
                     3            76.890833
                     4            76.496667
Medium     1         1            48.355833
                     2            46.313333
                     3            48.970417
                     4            47.050833
           2         1            40.018148
                     2            39.971481
                     3            39.532593
                     4            36.935185
           3         1            45.216552
                     2            46.879310
                     3            44.342414
                     4            45.437241
Small      1         1            58.676000
                     2            62.286000
                     3            59.078000
                     4            60.610000
           2         1            49.087500
                     2            51.870000
                     3            46.925000
                     4            55.360000
           3         1            58.991667
                     2            60.203333
                     3            57.856667
                     4            61.005000

In [48]:
# Feels like the promotion 1 and 3 does a better job compared to promotion 2

In [26]:
market_df.groupby("AgeOfStore").agg({"SalesInThousands":"mean"}).sort_values(by="SalesInThousands", ascending=False)

,SalesInThousands
AgeOfStore,
23,65.097500
19,63.638000
3,60.227500
20,60.202500
22,59.688333
13,59.642500
2,59.179500
1,58.415625
11,57.159375


In [42]:
#Maybe the age of store makes a difference in promotions, can be investigated further in the future

In [38]:
market_df.groupby("Promotion").agg({"SalesInThousands":["mean", "sum"]})

SalesInThousands          
                      mean       sum
Promotion                           
1                58.099012   9993.03
2                47.329415   8897.93
3                55.364468  10408.52

In [40]:
#Promotion types seem to have some differences, let's see if this is correct in statistical tests or not.

## 3) Test and Hypothesis Design

Business problem: We have three types of promotions. Do they have statistically different sale outcomes?

**Hypothesis:**

**H0:** P1 = P2 = P3 ***(Promotion types don't have a statistically significant difference in means)***

**H1:** P1 =! P2 =! P3 ***(At least one of the promotion types have a statistically significant difference in means)***

## 4) Assumption Tests

In [52]:
# If assumptions are correct: one way anova test
# If incorrect : kruskal test

In [60]:
#Let's start with normality assumption, using Shapiro-Wilk Test

In [56]:
from scipy.stats import shapiro

for promo in market_df["Promotion"].unique():
    sales = market_df[market_df["Promotion"] == promo]["SalesInThousands"]
    stat, p = shapiro(sales)
    print(f"Promotion {promo}: W={stat:.3f}, p-value={p:.4f}")

Promotion 3: W=0.921, p-value=0.0000
Promotion 2: W=0.915, p-value=0.0000
Promotion 1: W=0.915, p-value=0.0000


In [64]:
#Variance Homogenity with Levene's Test
from scipy.stats import levene

groups = [
    market_df[market_df["Promotion"] == promo]["SalesInThousands"]
    for promo in market_df["Promotion"].unique()
]

stat, p = levene(*groups)
print(f"Levene test p-value: {p:.4f}")

Levene test p-value: 0.2818


In [66]:
0.05 < 0.2818

True

All groups show statistically significant deviations from normality.
Our Levene's Test says that variances are unequal.

In this case, we can move on with Welch's ANOVA Test.

This test can be used when we have unequal variances, large sample sizes and robustness to non-normality.

## Welch's ANOVA Test

In [72]:
welch = pg.welch_anova(
    dv="SalesInThousands",
    between="Promotion",
    data=market_df
)

welch

,Source,ddof1,ddof2,F,p-unc,np2
0,Promotion,2,360.192581,23.224109,3.259974e-10,0.074557


In [78]:
3.259974e-10 < 0.05

True

Our p-value (p-unc) is lower than 0.05. In this case we can reject the null hypothesis and say that:

***Promotion types have a statistically significant effect on sales***

## Post-Hoc Test

In [91]:
##Post-Hoc Test
    #This type of test will allow us to see which promotion type is significantly different from others
    #Also it will give information about statistics compared to each other

In [89]:
posthoc = pg.pairwise_gameshowell(
    dv="SalesInThousands",
    between="Promotion",
    data=market_df
)

posthoc

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,1,2,58.099012,47.329415,10.769597,1.675542,6.427529,346.783619,1.286439e-09,0.679522
1,1,3,58.099012,55.364468,2.734544,1.757393,1.556022,355.918164,2.662448e-01,0.163744
2,2,3,47.329415,55.364468,-8.035053,1.646058,-4.881393,370.020530,4.684644e-06,-0.502467


## Results